In [ ]:
import numpy as np
import cv2
from PIL import Image
import pyautogui

class TemplateLibrary:
    def __init__(self):
        self.templates = {}

    def add_template(self, name, template_path):
        # Load the template image and add it to the dictionary
        try:
            template = np.asarray(Image.open(template_path))
        except FileNotFoundError:
            print(f"Error: Could not find file '{template_path}'")
            return
        gray_template = cv2.cvtColor(template, cv2.COLOR_BGR2GRAY)
        self.templates[name] = gray_template

  
    def find_template(self, gray_image, template_name):
        # Check if the template exists in the dictionary
        if template_name not in self.templates:
            print(f"Error: Template '{template_name}' not found in library")
            return []

        # Convert the screenshot to grayscale
        template = self.templates[template_name]

        # Find all matches using template matching
        result = cv2.matchTemplate(gray_image, template, cv2.TM_CCOEFF_NORMED)
        threshold = 0.9
        loc = np.where(result >= threshold)
        print(loc)
        matches = []
        for pt in zip(*loc[::-1]):
            # Compute the center of the match
            center_x = pt[0] + template.shape[1] // 2
            center_y = pt[1] + template.shape[0] // 2
            matches.append((center_x, center_y))

        return matches

    def find_all_templates(self, image):
        # Initialize an empty dictionary to store the results
        results = {}

        image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        # Find the center positions of all templates in the image
        for name, template in self.templates.items():
            centers = self.find_template(image, name)
            if centers:
                results[name] = centers

        return results

In [ ]:
# Example usage
library = TemplateLibrary()
library.add_template('small_image', 'small_image.png')

In [ ]:
from PIL import ImageGrab

# Capture the entire screen
screenshot = ImageGrab.grab()
image = np.asarray(screenshot)

In [ ]:
image_o = pyautogui.screenshot()
image = np.asarray(image_o)

image_o

In [ ]:
results = library.find_all_templates(image)
print(results)